In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')

In [3]:
# cell to run on cifar10_probs.csv (from resnet110)

# make df where column "label" is column 10 of data_cifar10_probs and column "output" is an array of the first 10 columns of data_cifar10_probs as a list of floats
data_cifar10_probs = pd.read_csv('notebooks/cifar10_probs.csv', sep=',', header=None)
data_cifar10_probs = data_cifar10_probs.rename(columns={10: "label"})
data_cifar10_probs["output"] = data_cifar10_probs.iloc[:, 0:10].values.tolist()
# make 

data_cifar10_probs = data_cifar10_probs.drop(data_cifar10_probs.columns[0:10], axis=1)
data_cifar10_probs["output"] = data_cifar10_probs["output"].astype(str)

data_cifar10_probs

,label,output
0,3,"[2.3661317e-09, 1.266867e-08, 8.9752067e-10, 0..."
1,8,"[2.0486328e-11, 8.667231e-08, 4.1953889e-16, 1..."
2,8,"[1.9216972e-07, 0.002469865, 3.1885104e-12, 3...."
3,0,"[0.999424, 1.3174234e-05, 4.9130163e-06, 2.804..."
4,6,"[3.4858956e-09, 1.7338067e-06, 1.1511221e-06, ..."
...,...,...
9995,8,"[0.20628296, 1.48559475e-05, 1.4134733e-06, 2...."
9996,3,"[4.3346837e-14, 1.0565218e-13, 8.082701e-10, 1..."
9997,5,"[8.1107293e-10, 9.9917585e-09, 7.729877e-08, 1..."
9998,1,"[1.5138195e-08, 0.99875677, 1.074252e-05, 3.15..."


In [90]:
# cell to run on labels_convolutional_cifar10.csv

# # we have to remember to turn " " into ","
# data1 = pd.read_csv('labels (2).csv', sep=';')
# # get types of columns
# type(data1["output"][0])

# we have to remember to turn " " into ","

In [4]:
smx = data_cifar10_probs['output']
labels = data_cifar10_probs['label'].astype(int)

# create an empty array
lista = np.array([])
for i in range(len(smx)):
    lista = np.append(lista, literal_eval(smx[i]))
    
smx = lista.reshape(len(smx), 10)

In [92]:
smx.shape

(10000, 10)

In [5]:
# Problem setup
n=500 # number of calibration points
alpha = 0.02 # 1-alpha is the desired coverage

In [6]:
# Split the softmax scores into calibration and validation sets (save the shuffling)

# n Trues e (smx.shape[0]-n) Falses
idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0 

# embaralha os Trues e Falses
np.random.shuffle(idx) 

# pega os valores de softmax de acordo com os Trues e Falses
cal_smx, val_smx = smx[idx,:], smx[~idx, :] 
#val_smx = val_smx[2].reshape(1,10) #use to have 1 img to predict


# pega os valores de labels de acordo com os Trues e Falses
cal_labels, val_labels = np.array(labels[idx]), np.array(labels[~idx])
#cal_labels, val_labels = np.array(labels[idx]), np.array(labels[~idx])[2] #use to have 1 img to predict

In [95]:
cal_smx.shape, val_smx.shape

((500, 10), (9500, 10))

In [96]:
cal_labels.shape, val_labels.shape

((500,), (9500,))

### Conformal prediction happens here

In [12]:

cal_scores = 1 - cal_smx[np.arange(n), cal_labels]
alphas = []
i =0 
for image in val_smx:
    
    # try alphas until there's only one class in the prediction set
    for alpha in np.arange(0.01, 1, 0.0001):
        q_level = np.ceil((n + 1) * (1 - alpha)) / n
        qhat = np.quantile(cal_scores, q_level, interpolation='higher')

        prediction_sets = [image] >= (1 - qhat)  # 3: form prediction sets

        if np.sum(prediction_sets) == 1:
            alphas.append(alpha)
            break  # exit the inner loop if an alpha is foundz

In [8]:
len(alphas)

9482

In [10]:
# create fake logits list (1-alphas)
fake_logits = 1 - np.array(alphas)
print(fake_logits)

[0.99  0.99  0.978 ... 0.99  0.98  0.99 ]


In [78]:
# 1: get conformal scores. n = calib_Y.shape[0]

# criamos um vetor cal_scores com (1 -(probabilidade atribuida pelo modelo de que a imagem tenha seu label verdadeiro))
# cal score é quanto menor, melhor
cal_scores = 1-cal_smx[np.arange(n),cal_labels]

# 2: get adjusted quantile
# qhat será o valor de s_i (entrada de cal_scores) que limita os 1-alpha menores scores (os melhores!)
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, interpolation='higher')

prediction_sets = val_smx >= (1-qhat) # 3: form prediction sets

In [79]:
empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),val_labels].mean()
print(f"The empirical coverage is: {empirical_coverage}")

The empirical coverage is: 0.28378947368421054


In [80]:
prediction_sets.shape

(9500, 10)

In [81]:
val_labels.shape

(9500,)

In [82]:
# prediction_sets_one_guess = prediction_set removing all lists with only one guess
prediction_sets_many_guesses = []
for i in range(len(prediction_sets)):
    if prediction_sets[i].sum() != 1:
        prediction_sets_many_guesses.append(prediction_sets[i])
    else:
        # make val_labels[i]==-1
        val_labels[i] = -1

# remove all -1 from val_labels
val_labels = val_labels[val_labels != -1]

# get the accuracy of the model
accuracy = 0
for i in range(len(prediction_sets_many_guesses)):
    if prediction_sets_many_guesses[i][val_labels[i]] == True:
        accuracy += 1
print(accuracy)
print(len(prediction_sets_many_guesses))

0
6802


In [83]:
# prediction_sets_one_guess = prediction_set removing all lists with more than one guess
prediction_sets_one_guess = []
for i in range(len(prediction_sets)):
    if prediction_sets[i].sum() == 1:
        prediction_sets_one_guess.append(prediction_sets[i])
    else:
        # make val_labels[i]==-1
        val_labels[i] = -1

# remove all -1 from val_labels
val_labels = val_labels[val_labels != -1]

# get the accuracy of the model
accuracy = 0
for i in range(len(prediction_sets_one_guess)):
    if prediction_sets_one_guess[i][val_labels[i]] == True:
        accuracy += 1
print(accuracy)
print(len(prediction_sets_one_guess))

IndexError: index 6802 is out of bounds for axis 0 with size 6802

In [ ]:
# count how many times there is more than one label in the prediction set
count = 0
for i in range(len(prediction_sets)):
    if sum(prediction_sets[i]) > 9:
        count += 1
print(f"The number of times there is more than one label in the prediction set is: {count}")

The number of times there is more than one label in the prediction set is: 0
